In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf

%load_ext autoreload
%autoreload 2

In [2]:
os.chdir('..')

In [4]:
from IndoEQ.IndoEQ_utils import IndoEQ
from IndoEQ.EqT_utils import EqTransformerCopy
from IndoEQ.LEQNet_utils import LEQNetCopy

inp = tf.keras.Input(shape=(6000, 3))
model_eqt = EqTransformerCopy()(inp)
model_eqt.summary()

Learning rate:  0.001
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 6000, 3)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 6000, 8)      272         ['input_1[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 3000, 8)      0           ['conv1d[0][0]']                 
                                                                                                  
 conv1d_1 (Conv1D)              (None, 3000, 16)     1168        ['max_pooling1d[0][0]']          
                                                                        

In [5]:
model_leq = LEQNetCopy()(inp)
model_leq.summary() 

NameError: name 'LEQNetCopy' is not defined

In [6]:
model_indoeq = IndoEQ(use_prelu=False)(inp)
model_indoeq.summary()

NameError: name 'IndoEQ' is not defined

In [6]:
import h5py

csv_path = os.path.join('data','2015.2023.20.5.metadata_indo.csv')
h5_path = os.path.join('data','2015.2023.20.5.chunk_indo.hdf5')

# csv_path = os.path.join('data','chunk2_ev.1069.csv')
# h5_path = os.path.join('data','chunk2_ev.1069.hdf5')
metadata = pd.read_csv(csv_path)
ev_list = metadata['trace_name'].to_list()

h5 = h5py.File(h5_path, 'r')

In [7]:
# TIMESTEPS = 6000

# X, label_p, label_s, label_eq = [], [], [], []

# for ev in ev_list:
#     dset = h5.get(f'data/{ev}')
#     if dset is None:
#         continue

#     X.append(np.array(dset))
#     t = np.arange(start=0., stop=1., step=0.05)

#     y_ = np.zeros((TIMESTEPS,))
#     time_p = dset.attrs['p_arrival_sample'].astype(int)
#     y_[time_p-20:time_p] = t
#     y_[time_p] = 1.
#     y_[time_p+1:time_p+21] = t[::-1]
#     label_p.append(y_)

#     y_ = np.zeros((TIMESTEPS,))
#     time_s = dset.attrs['s_arrival_sample'].astype(int)
#     y_[time_s-20:time_s] = t
#     y_[time_s] = 1.
#     y_[time_s+1:time_s+21] = t[::-1]
#     label_s.append(y_)
        
#     y_ = np.zeros((TIMESTEPS,))
#     time_eq_end = int(time_s + 1.4 * (time_s - time_p))
#     y_[time_p:time_eq_end] = 1.0
#     label_eq.append(y_)

# X = np.array(X)
# label_p = np.array(label_p)
# label_s = np.array(label_s)
# label_eq = np.array(label_eq)

# from sklearn.model_selection import train_test_split

# X_train, X_test, label_p_train, label_p_test, label_s_train, label_s_test, \
#     label_eq_train, label_eq_test = train_test_split(
#         X, label_p, label_s, label_eq, test_size=X.shape[0] - 10, random_state=42
#     )

In [46]:
model_eqt.fit(X_train, [label_eq_train, label_p_train, label_s_train], epochs=1, batch_size=32)

1/1 [==============================] - 564s 564s/step - loss: 2.6389 - detector_loss: 0.6875 - picker_P_loss: 0.6925 - picker_S_loss: 0.6883 - detector_f1: 0.2426 - picker_P_f1: 4.2157e-04 - picker_S_f1: 0.0000e+00


In [47]:
model_leq.fit(X_train, [label_eq_train, label_p_train, label_s_train], epochs=1, batch_size=32)

1/1 [==============================] - 356s 356s/step - loss: 0.6932 - detector_loss: 0.6931 - picker_P_loss: 0.6932 - picker_S_loss: 0.6932 - detector_f1: 0.1513 - picker_P_f1: 3.3228e-04 - picker_S_f1: 3.1841e-04


In [6]:
import h5py

csv_path = os.path.join('data','2015.2023.20.5.metadata_indo.csv')
h5_path = os.path.join('data','2015.2023.20.5.chunk_indo.hdf5')
metadata = pd.read_csv(csv_path)
# ev_list = metadata['trace_name'].to_list()

h5 = h5py.File(h5_path, 'r')

In [9]:
unmatched_data = []

ev_list = list(h5.keys())
for ev in ev_list:
    dset = h5.get(f'{ev}')
    e, n, z = dset.get('data')
    if e.shape[0] != 6000:
        unmatched_data.append(ev)

print(len(unmatched_data))

6410


In [17]:
ev = ev_list[0]
dset = h5.get(f'{ev}')
data = np.array(dset.get('data'))
output = np.array(dset.get('output'))

data.T[:6000].shape

(6000, 3)

In [96]:
dset = h5.get(f'{ev}/data')
dset.attrs['s_']

2796

In [17]:
from IndoEQ.trainer import trainer

trainer(input_hdf5=h5_path,
        input_csv='data/2015.2023.20.5.metadata_indo.csv',
        output_name='test_indoeq',
        model_class='indoeq',
        use_prelu=True,
        input_dimension=(6000, 3),
        cnn_blocks=5,
        lstm_blocks=2,
        padding='same',
        activation='relu',
        drop_rate=0.1,
        shuffle=True,
        label_type='triangle',
        normalization_mode='std',
        augmentation=False,
        loss_weights=[0.05, 0.40, 0.55],
        loss_types=['binary_crossentropy', 'binary_crossentropy', 'binary_crossentropy'],
        train_valid_test_split=[0.8, 0.1, 0.1],
        mode='generator',
        batch_size=8,
        epochs=1,
        monitor='val_loss',
        patience=10)

Learning rate:  0.001
Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 6000, 3)]    0           []                               
                                                                                                  
 separable_conv1d_64 (Separable  (None, 6000, 8)     65          ['input[0][0]']                  
 Conv1D)                                                                                          
                                                                                                  
 max_pooling1d_55 (MaxPooling1D  (None, 2000, 8)     0           ['separable_conv1d_64[0][0]']    
 )                                                                                                
                                                                      

KeyboardInterrupt: 